# **Utility-Based Agents**
In intelligent agent design, **utility-based agents** represent a significant leap beyond simple goal achievement. While goal-based agents ask “Can I reach the goal?”, utility-based agents ask “Which path to the goal is best?” They evaluate options not just by feasibility, but by **preference**, encoded through a **utility function.**

This notebook demonstrates how different agents can make rational yet divergent choices when faced with identical options—because each has its own internal “personality” or utility function. We’ll explore this through a travel scenario where agents must choose among routes that vary in time, cost, safety, and scenic value.

Let’s begin by defining what a route looks like in our world.

# **1. The Route Class: Modeling Options in the Environment**
Each possible action (in this case, a travel route) is represented as a `Route` object with four key attributes:

- `time_minutes:` Lower is better (faster is preferred).
- `cost_dollars:` Lower is better (cheaper is preferred).
- `safety_rating:` Higher is better (rated 1–10).
- `view_rating:` Higher is better (rated 1–10).

These multi-dimensional features allow us to model complex trade-offs that real-world decisions often involve.

In [ ]:
import time

class Route:
    """
    Represents a possible path to the goal.
    It has multiple attributes, not just 'distance'.
    """
    def __init__(self, name, time_minutes, cost_dollars, safety_rating, view_rating):
        self.name = name
        self.time = time_minutes      # Lower is better
        self.cost = cost_dollars      # Lower is better
        self.safety = safety_rating   # Higher is better (1-10)
        self.view = view_rating       # Higher is better (1-10)

    def __repr__(self):
        return (f"{self.name}: {self.time}min, ${self.cost}, "
                f"Safety: {self.safety}/10, View: {self.view}/10")

# **2. The Utility-Based Agent: Rational Choice Through Scoring**
The core of a utility-based agent is its **utility function**, which translates a complex option into a single numerical score, a “happiness metric.” This score allows the agent to rank alternatives objectively, even when they differ along multiple dimensions.

**Key Concepts:**
- **The weights Dictionary** (The "Personality"): This represents the Utility Function.

  - For the Business Agent, time has a large negative weight (-2.0). This means every minute lost hurts its score significantly.

  - For the Student Agent, cost has a large negative weight (-5.0). Paying $1 hurts the student more than wasting 1 minute.
- **utility_function:** This is the mathematical core. It reduces a complex object (a Route with time, cost, view) down to a single number (a score).

Equation: $$U(s)=w_{1}f_{1} +w_{2}f_{2}+...$$

- **Decision Process:** Evaluate all routes, then select the one with the highest utility score.

In [ ]:
class UtilityBasedAgent:
    def __init__(self, name, weights):
        """
        The 'weights' dictionary defines the Personality/Utility Function of the agent.
        Negative weights mean we want to minimize that factor (like cost).
        Positive weights mean we want to maximize it (like safety).
        """
        self.name = name
        self.weights = weights

    def utility_function(self, route):
        """
        Calculates the 'Happiness Score' (Utility) for a specific route.
        Formula: Sum(Attribute * Weight)
        """
        u = 0
        u += route.time * self.weights.get('time', 0)
        u += route.cost * self.weights.get('cost', 0)
        u += route.safety * self.weights.get('safety', 0)
        u += route.view * self.weights.get('view', 0)
        return u

    def choose_route(self, routes):
        """
        Evaluates all options and picks the one with the highest Utility.
        """
        print(f"\n--- {self.name} is thinking... ---")
        best_route = None
        highest_utility = float('-inf')

        for route in routes:
            score = self.utility_function(route)
            print(f"Evaluating '{route.name}': Utility Score = {score:.1f}")

            if score > highest_utility:
                highest_utility = score
                best_route = route

        print(f"Decision: {self.name} chooses '{best_route.name}'")
        return best_route

# **Simulation Setup: Defining the World and Agents**
We now define the environment and create two distinct agents with very different priorities.

**Available Routes:**
- **Highway:** Fast and safe, but expensive and boring.
- **Back Roads:** Free and scenic, but slow and less safe.
- **City Center:** A middle-ground option on all fronts.

**Agent Personalities:**
- **Business Agent:** Hates wasting time (`time: -2.0`), doesn’t care about cost (`cost: -0.1`), and values safety (`safety: 2.0`).
- **Student Agent:** Hates spending money (`cost: -5.0`), doesn’t mind extra time (`time: -0.1`), and enjoys nice views (`view: 2.0`).

In [ ]:
# --- SIMULATION ---

# 1. Define the Environment (The possible solutions)
# A Goal-Based agent would consider all of these valid because they all reach the destination.
possible_routes = [
    Route("Highway",      time_minutes=20, cost_dollars=15, safety_rating=8, view_rating=2),
    Route("Back Roads",   time_minutes=45, cost_dollars=0,  safety_rating=6, view_rating=9),
    Route("City Center",  time_minutes=30, cost_dollars=5,  safety_rating=4, view_rating=5)
]

print("Available Routes to Destination:")
for r in possible_routes:
    print(f" - {r}")

# 2. Create Agent A: "The Busy Business Executive"
# Utility: Hates wasting time (-10), doesn't care about cost (-1), wants safety (+5)
exec_weights = {'time': -2.0, 'cost': -0.1, 'safety': 2.0, 'view': 0.0}
agent_exec = UtilityBasedAgent("Business Agent", exec_weights)

# 3. Create Agent B: "The Broke Student"
# Utility: Hates spending money (-10), doesn't mind time (-0.1), likes views (+2)
student_weights = {'time': -0.1, 'cost': -5.0, 'safety': 1.0, 'view': 2.0}
agent_student = UtilityBasedAgent("Student Agent", student_weights)

# 4. Run Decisions
agent_exec.choose_route(possible_routes)
agent_student.choose_route(possible_routes)

Available Routes to Destination:
 - Highway: 20min, $15, Safety: 8/10, View: 2/10
 - Back Roads: 45min, $0, Safety: 6/10, View: 9/10
 - City Center: 30min, $5, Safety: 4/10, View: 5/10

--- Business Agent is thinking... ---
Evaluating 'Highway': Utility Score = -25.5
Evaluating 'Back Roads': Utility Score = -78.0
Evaluating 'City Center': Utility Score = -52.5
Decision: Business Agent chooses 'Highway'

--- Student Agent is thinking... ---
Evaluating 'Highway': Utility Score = -65.0
Evaluating 'Back Roads': Utility Score = 19.5
Evaluating 'City Center': Utility Score = -14.0
Decision: Student Agent chooses 'Back Roads'


Back Roads: 45min, $0, Safety: 6/10, View: 9/10

Notice that both agents are acting rationally. The student isn't "wrong" for taking the slow road; they are optimizing for their specific utility function (saving money). This illustrates a foundational principle of utility theory: **rationality is defined relative to an agent’s preferences**, not an absolute standard.



# **Summary: Why Utility Matters**
This example highlights three critical ideas in advanced agent design:

- **Rational ≠ Identical:** Two agents can be perfectly rational yet make opposite choices because they value outcomes differently.
- **Explicit Preferences:** The utility function makes an agent’s priorities transparent and tunable—essential for debugging and alignment.
- **Scalable Decision-Making:** By reducing complex, multi-attribute choices to a single score, utility functions enable efficient comparison even among hundreds of options.